# 🧩 Contrôle Qualité - Backtester Modifié

Ce notebook effectue une vérification rapide après les modifications apportées au `strategy_backtester.py` pour s'assurer que :

1. **Période temporelle** : Les données couvrent bien ~5 ans (fin 2020 → 2025-12-08)
2. **Volume de données** : ~68k courses et ~661k partants comme attendu
3. **Nombre de paris** : Largement supérieur aux 2,953 paris de la période limitée
4. **Statistiques clés** : Taux de victoire ~8-10%, distribution des cotes et EV

## 1. Chargement des données et configuration

In [1]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path

# Ajouter le chemin du projet pour importer les modules
sys.path.append(".")

# Importer les fonctions du backtester modifié
from strategy_backtester import load_real_predictions, BacktestConfig

print("📦 Bibliothèques importées avec succès")
print("✅ Fonctions du backtester chargées")

📦 Bibliothèques importées avec succès
✅ Fonctions du backtester chargées


In [2]:
# Configuration pour le test - période complète sur 5 ans
config = BacktestConfig(
    train_start="2020-11-27",
    train_end="2023-06-30",
    val_start="2023-07-01",
    val_end="2023-08-31",
    test_start="2020-11-27",  # Toute la période pour le contrôle
    test_end="2025-12-08",
    value_cutoff=0.05,
)

print("⚙️ Configuration créée:")
print(f"   Test période: {config.test_start} → {config.test_end}")
print(f"   Value cutoff: {config.value_cutoff}")

⚙️ Configuration créée:
   Test période: 2020-11-27 → 2025-12-08
   Value cutoff: 0.05


In [3]:
# Chargement des vraies prédictions calibrées
print("📂 Chargement des prédictions calibrées...")
df = load_real_predictions(config=config)

print("\n✅ Données chargées avec succès:")
print(f"   Shape: {df.shape}")
print(f"   Colonnes: {list(df.columns)}")

📂 Chargement des prédictions calibrées...
📂 Chargement des prédictions depuis: data/backtest_predictions_calibrated.csv
   Données brutes: 661,105 entrées
   Après filtre temporel 2020-11-27 → 2025-12-08: 661,105 entrées
   Après nettoyage des NaN: 661,105 entrées
   Période finale: 2020-11-27 00:00:00 → 2025-12-08 00:00:00
   Courses: 68,216

✅ Données chargées avec succès:
   Shape: (661105, 12)
   Colonnes: ['race_key', 'id_cheval', 'date_course', 'p_model_win', 'p_model_norm', 'p_calibrated', 'p_final', 'is_win', 'place', 'position_arrivee', 'cote_sp', 'split']


## 2. Vérification de la période temporelle

**Attendu :** 
- Min ≈ fin 2020 (2020-11-27)
- Max ≈ 2025-12-08
- Période couvrant ~5 années complètes

In [4]:
# Vérification des dates
date_min = df["date_course"].min()
date_max = df["date_course"].max()
duree_jours = (date_max - date_min).days
duree_annees = duree_jours / 365.25

print("📅 PÉRIODE TEMPORELLE:")
print(f"   Date min: {date_min}")
print(f"   Date max: {date_max}")
print(f"   Durée: {duree_jours:,} jours ({duree_annees:.1f} années)")

# Validation
print("\n✅ VALIDATION:")
if date_min.year <= 2020 and date_max.year >= 2025:
    print("   ✓ Période couvre bien ~5 ans (2020-2025)")
else:
    print(f"   ⚠️ Période inattendue: {date_min.year}-{date_max.year}")

if duree_annees >= 4.0:
    print(f"   ✓ Durée suffisante: {duree_annees:.1f} années")
else:
    print(f"   ⚠️ Durée insuffisante: {duree_annees:.1f} années")

📅 PÉRIODE TEMPORELLE:
   Date min: 2020-11-27 00:00:00
   Date max: 2025-12-08 00:00:00
   Durée: 1,837 jours (5.0 années)

✅ VALIDATION:
   ✓ Période couvre bien ~5 ans (2020-2025)
   ✓ Durée suffisante: 5.0 années


## 3. Analyse du volume de courses et partants

**Attendu :** 
- ~68,000 courses uniques (race_key)
- ~661,000 partants total
- Cohérent avec l'audit AUDIT_SYSTEME_ANALYSE_V2

In [5]:
# Analyse des volumes
n_courses = df["race_key"].nunique()
n_partants = len(df)
moyenne_partants = df.groupby("race_key").size().mean()

print("📊 VOLUME DE DONNÉES:")
print(f"   Courses uniques: {n_courses:,}")
print(f"   Total partants: {n_partants:,}")
print(f"   Moyenne partants/course: {moyenne_partants:.1f}")

# Validation contre les valeurs attendues
print("\n✅ VALIDATION:")
if 60_000 <= n_courses <= 80_000:
    print("   ✓ Nombre de courses dans la plage attendue (~68k)")
else:
    print(f"   ⚠️ Nombre de courses inattendu: {n_courses:,} (attendu ~68k)")

if 600_000 <= n_partants <= 700_000:
    print("   ✓ Nombre de partants dans la plage attendue (~661k)")
else:
    print(f"   ⚠️ Nombre de partants inattendu: {n_partants:,} (attendu ~661k)")

# Répartition par année pour vérifier la couverture
print("\n📈 RÉPARTITION PAR ANNÉE:")
repartition = (
    df.groupby(df["date_course"].dt.year)
    .agg({"race_key": "nunique", "id_cheval": "count"})
    .rename(columns={"race_key": "courses", "id_cheval": "partants"})
)

for year, row in repartition.iterrows():
    print(f"   {year}: {row['courses']:,} courses, {row['partants']:,} partants")

📊 VOLUME DE DONNÉES:
   Courses uniques: 68,216
   Total partants: 661,105
   Moyenne partants/course: 9.7

✅ VALIDATION:
   ✓ Nombre de courses dans la plage attendue (~68k)
   ✓ Nombre de partants dans la plage attendue (~661k)

📈 RÉPARTITION PAR ANNÉE:
   2020: 1,021 courses, 10,429 partants
   2021: 13,085 courses, 126,889 partants
   2022: 13,455 courses, 129,463 partants
   2023: 13,461 courses, 129,421 partants
   2024: 13,951 courses, 136,535 partants
   2025: 13,243 courses, 128,368 partants


## 4. Vérification du nombre de paris potentiels

**Objectif :** Vérifier que le nombre de paris potentiels est **largement supérieur** aux 2,953 paris de la période limitée.

Calcul de l'EV et application du filtre value_cutoff.

In [6]:
# Calcul de l'Expected Value
df["ev"] = df["p_final"] * df["cote_sp"] - 1.0

# Statistiques sur l'EV
print("💰 EXPECTED VALUE (EV):")
print(f"   EV moyenne: {df['ev'].mean():.4f}")
print(f"   EV médiane: {df['ev'].median():.4f}")
print(f"   EV min: {df['ev'].min():.4f}")
print(f"   EV max: {df['ev'].max():.4f}")

# Nombre de value bets selon différents cutoffs
cutoffs = [0.03, 0.05, 0.08, 0.10]
print("\n🎯 VALUE BETS selon cutoff:")

for cutoff in cutoffs:
    n_value_bets = (df["ev"] >= cutoff).sum()
    pourcentage = (n_value_bets / len(df)) * 100
    print(f"   EV >= {cutoff:0.2f}: {n_value_bets:,} paris ({pourcentage:.1f}%)")

# Validation : doit être >> 2,953 pour cutoff 0.05
n_value_default = (df["ev"] >= 0.05).sum()
print(f"\n✅ VALIDATION (cutoff 0.05 = {n_value_default:,} paris):")
if n_value_default > 10000:
    print("   ✓ Largement supérieur aux 2,953 paris de la période limitée")
elif n_value_default > 2953:
    print(f"   ✓ Supérieur aux 2,953 paris (x{n_value_default/2953:.1f})")
else:
    print(f"   ⚠️ Pas assez de value bets: {n_value_default:,} <= 2,953")

💰 EXPECTED VALUE (EV):
   EV moyenne: -0.2125
   EV médiane: -0.3598
   EV min: -0.9295
   EV max: 82.1299

🎯 VALUE BETS selon cutoff:
   EV >= 0.03: 145,840 paris (22.1%)
   EV >= 0.05: 139,321 paris (21.1%)
   EV >= 0.08: 129,901 paris (19.6%)
   EV >= 0.10: 124,117 paris (18.8%)

✅ VALIDATION (cutoff 0.05 = 139,321 paris):
   ✓ Largement supérieur aux 2,953 paris de la période limitée


## 5. Calcul des statistiques de performance rapides

**Validation :** Taux de victoire moyen ~8-10% selon AUDIT_SYSTEME_ANALYSE_V2

In [7]:
# Statistiques clés de performance
taux_victoire_reel = df["is_win"].mean()
taux_victoire_predit = df["p_final"].mean()

print("🏆 TAUX DE VICTOIRE:")
print(f"   Réel (is_win): {taux_victoire_reel:.3f} ({taux_victoire_reel*100:.1f}%)")
print(f"   Prédit (p_final): {taux_victoire_predit:.3f} ({taux_victoire_predit*100:.1f}%)")

# Validation contre les stats attendues
print("\n✅ VALIDATION:")
if 0.08 <= taux_victoire_reel <= 0.12:
    print("   ✓ Taux de victoire réel dans la plage attendue (8-12%)")
else:
    print(f"   ⚠️ Taux de victoire réel inattendu: {taux_victoire_reel*100:.1f}%")

if 0.08 <= taux_victoire_predit <= 0.12:
    print("   ✓ Taux de victoire prédit cohérent (8-12%)")
else:
    print(f"   ⚠️ Taux de victoire prédit inattendu: {taux_victoire_predit*100:.1f}%")

# Calibration rapide
diff_calibration = abs(taux_victoire_reel - taux_victoire_predit)
print(f"   📊 Écart calibration: {diff_calibration:.3f} ({diff_calibration*100:.1f}pp)")

if diff_calibration <= 0.005:
    print("   ✓ Excellente calibration (écart <= 0.5pp)")
elif diff_calibration <= 0.01:
    print("   ✓ Bonne calibration (écart <= 1.0pp)")
else:
    print("   ⚠️ Calibration à améliorer (écart > 1.0pp)")

🏆 TAUX DE VICTOIRE:
   Réel (is_win): 0.103 (10.3%)
   Prédit (p_final): 0.103 (10.3%)

✅ VALIDATION:
   ✓ Taux de victoire réel dans la plage attendue (8-12%)
   ✓ Taux de victoire prédit cohérent (8-12%)
   📊 Écart calibration: 0.000 (0.0pp)
   ✓ Excellente calibration (écart <= 0.5pp)


## 6. Analyse des cotes et Expected Value

Distribution complète des cotes et EV pour vérifier la cohérence des données.

In [8]:
# Distribution des cotes
print("📊 DISTRIBUTION DES COTES (cote_sp):")
print(df["cote_sp"].describe())

print("\n📊 DISTRIBUTION DE L'EXPECTED VALUE:")
print(df["ev"].describe())

# Vérifications de cohérence
print("\n🔍 VÉRIFICATIONS DE COHÉRENCE:")

# Cotes dans des plages raisonnables
cotes_extremes = ((df["cote_sp"] < 1.1) | (df["cote_sp"] > 100)).sum()
print(f"   Cotes extrêmes (<1.1 ou >100): {cotes_extremes:,} ({cotes_extremes/len(df)*100:.1f}%)")

# EV distribution
ev_positifs = (df["ev"] > 0).sum()
print(f"   EV positifs: {ev_positifs:,} ({ev_positifs/len(df)*100:.1f}%)")

# Probabilités dans [0,1]
prob_invalides = ((df["p_final"] < 0) | (df["p_final"] > 1)).sum()
print(f"   Probabilités invalides: {prob_invalides:,}")

if prob_invalides == 0 and cotes_extremes < len(df) * 0.01:  # < 1% de valeurs extrêmes
    print("\n✅ Données cohérentes pour le backtesting")
else:
    print("\n⚠️ Attention aux données extrêmes avant backtesting")

📊 DISTRIBUTION DES COTES (cote_sp):
count    661105.000000
mean         18.868093
std          26.399332
min           1.050000
25%           5.000000
50%          10.000000
75%          22.000000
max         999.000000
Name: cote_sp, dtype: float64

📊 DISTRIBUTION DE L'EXPECTED VALUE:
count    661105.000000
mean         -0.212542
std           0.624214
min          -0.929515
25%          -0.591580
50%          -0.359774
75%          -0.024417
max          82.129920
Name: ev, dtype: float64

🔍 VÉRIFICATIONS DE COHÉRENCE:
   Cotes extrêmes (<1.1 ou >100): 11,604 (1.8%)
   EV positifs: 156,213 (23.6%)
   Probabilités invalides: 0

⚠️ Attention aux données extrêmes avant backtesting


## 💡 Résumé du contrôle qualité

Si tous les checks passent :
- ✅ Période : ~5 ans (2020-2025)  
- ✅ Volume : ~68k courses, ~661k partants
- ✅ Paris potentiels : >> 2,953 (largement supérieur)
- ✅ Taux de victoire : ~8-10% (cohérent)
- ✅ Données calibrées prêtes pour backtest

➡️ **Le backtester est prêt pour tester sur 5 années complètes !**